# Chapter 34: TypeScript with Vue.js

Vue.js has embraced TypeScript wholeheartedly in version 3, with the Composition API providing first-class TypeScript support. The introduction of `<script setup>` syntax has made TypeScript integration more seamless than ever. This chapter covers everything from basic setup to advanced patterns for building type-safe Vue applications.

---

## 34.1 Setting Up Vue with TypeScript

Modern Vue 3 projects use Vite by default, which provides excellent TypeScript support out of the box.

### Project Initialization with Vite

```bash
# Create a new Vue + TypeScript project
npm create vue@latest my-vue-app

# Follow prompts:
# ✔ TypeScript? … Yes
# ✔ JSX Support? … No (or Yes if needed)
# ✔ Vue Router? … Yes
# ✔ Pinia? … Yes
# ✔ Vitest? … Yes
# ✔ Cypress? … No (or Yes for E2E)
# ✔ ESLint? … Yes
# ✔ Prettier? … Yes

cd my-vue-app
npm install
npm run dev
```

**Project Structure:**
```
my-vue-app/
├── src/
│   ├── assets/           # Static assets
│   ├── components/       # Vue components
│   ├── composables/      # Shared composition functions
│   ├── router/           # Vue Router configuration
│   ├── stores/           # Pinia stores
│   ├── types/            # Global type definitions
│   ├── App.vue           # Root component
│   └── main.ts           # Entry point
├── env.d.ts              # Environment type declarations
├── tsconfig.json         # TypeScript configuration
├── vite.config.ts        # Vite configuration
└── package.json
```

**tsconfig.json Configuration:**
```json
{
  "compilerOptions": {
    "target": "ES2020",
    "useDefineForClassFields": true,
    "module": "ESNext",
    "lib": ["ES2020", "DOM", "DOM.Iterable"],
    "skipLibCheck": true,
    "moduleResolution": "bundler",
    "allowImportingTsExtensions": true,
    "resolveJsonModule": true,
    "isolatedModules": true,
    "noEmit": true,
    "jsx": "preserve",
    "strict": true,
    "noUnusedLocals": true,
    "noUnusedParameters": true,
    "noFallthroughCasesInSwitch": true,
    "baseUrl": ".",
    "paths": {
      "@/*": ["./src/*"],
      "@components/*": ["./src/components/*"],
      "@composables/*": ["./src/composables/*"],
      "@stores/*": ["./src/stores/*"]
    }
  },
  "include": ["src/**/*.ts", "src/**/*.tsx", "src/**/*.vue"],
  "references": [{ "path": "./tsconfig.node.json" }]
}
```

**vite.config.ts with TypeScript:**
```typescript
import { defineConfig } from 'vite'
import vue from '@vitejs/plugin-vue'
import { fileURLToPath, URL } from 'node:url'

export default defineConfig({
  plugins: [
    vue({
      script: {
        defineModel: true, // Enable experimental defineModel
        propsDestructure: true // Enable props destructure
      }
    }),
  ],
  resolve: {
    alias: {
      '@': fileURLToPath(new URL('./src', import.meta.url)),
      '@components': fileURLToPath(new URL('./src/components', import.meta.url)),
      '@composables': fileURLToPath(new URL('./src/composables', import.meta.url)),
      '@stores': fileURLToPath(new URL('./src/stores', import.meta.url))
    }
  }
})
```

**env.d.ts for TypeScript:**
```typescript
/// <reference types="vite/client" />

declare module '*.vue' {
  import type { DefineComponent } from 'vue'
  const component: DefineComponent<{}, {}, any>
  export default component
}

// Environment variables
interface ImportMetaEnv {
  readonly VITE_API_URL: string
  readonly VITE_APP_TITLE: string
  readonly VITE_DEBUG: string
}

interface ImportMeta {
  readonly env: ImportMetaEnv
}
```

---

## 34.2 Composition API with TypeScript

The Composition API is the recommended way to use TypeScript in Vue 3. It provides better type inference and explicit type control compared to the Options API.

### ref and reactive

```vue
<script setup lang="ts">
import { ref, reactive, computed, watch } from 'vue'

// ref with explicit type
const count = ref<number>(0)
const message = ref<string>('Hello')
const user = ref<{ name: string; age: number } | null>(null)

// ref with type inference (recommended)
const count2 = ref(0) // Type: Ref<number>
const name = ref('John') // Type: Ref<string>
const isActive = ref(true) // Type: Ref<boolean>

// reactive with interfaces
interface User {
  id: number
  name: string
  email: string
  preferences: {
    theme: 'light' | 'dark'
    notifications: boolean
  }
}

const userState = reactive<User>({
  id: 1,
  name: 'John Doe',
  email: 'john@example.com',
  preferences: {
    theme: 'light',
    notifications: true
  }
})

// Accessing reactive properties (no .value needed)
console.log(userState.name)
userState.preferences.theme = 'dark'

// ref vs reactive comparison
const state = reactive({
  count: 0,
  name: 'Vue'
})

// To replace entire object, use ref
const userRef = ref<User>({
  id: 1,
  name: 'Jane',
  email: 'jane@example.com',
  preferences: {
    theme: 'dark',
    notifications: false
  }
})

// Can replace entire ref value
userRef.value = {
  id: 2,
  name: 'Bob',
  email: 'bob@example.com',
  preferences: {
    theme: 'light',
    notifications: true
  }
}

// Cannot replace reactive object directly
// state = { count: 1, name: 'React' } // Error!

// Computed with types
const doubleCount = computed<number>(() => count.value * 2)

const fullName = computed({
  get: (): string => `${userState.name} (${userState.email})`,
  set: (newValue: string) => {
    // Parse and update
    const [name] = newValue.split(' ')
    userState.name = name
  }
})

// Watch with typed parameters
watch(count, (newVal: number, oldVal: number) => {
  console.log(`Count changed from ${oldVal} to ${newVal}`)
})

// Watch multiple sources
watch([count, () => userState.name], ([newCount, newName], [oldCount, oldName]) => {
  console.log('Multiple sources changed')
})
</script>

<template>
  <div>
    <p>Count: {{ count }}</p>
    <p>Double: {{ doubleCount }}</p>
    <button @click="count++">Increment</button>
    
    <p>User: {{ userState.name }}</p>
    <p>Theme: {{ userState.preferences.theme }}</p>
  </div>
</template>
```

**Explanation:**
- `ref<T>()` creates a reactive reference with type `Ref<T>`
- Access ref values with `.value` in TypeScript
- `reactive<T>()` creates reactive object of type `T`
- Access reactive properties directly (no `.value`)
- `computed()` infers return type automatically, or accepts explicit generic
- `watch()` callbacks receive properly typed parameters

### Lifecycle Hooks

```vue
<script setup lang="ts">
import { 
  onMounted, 
  onUpdated, 
  onUnmounted, 
  onBeforeMount,
  onBeforeUpdate,
  onBeforeUnmount,
  onErrorCaptured 
} from 'vue'

// Component instance type
import type { ComponentPublicInstance } from 'vue'

// onMounted with async operations
onMounted(async () => {
  console.log('Component mounted')
  await fetchData()
})

onUpdated(() => {
  console.log('Component updated')
})

onUnmounted(() => {
  console.log('Component unmounted')
  // Cleanup logic
})

// Error handling
onErrorCaptured((err: unknown, instance: ComponentPublicInstance | null, info: string) => {
  console.error('Error captured:', err)
  console.error('Component:', instance)
  console.error('Info:', info)
  return false // Prevent error from propagating
})

async function fetchData(): Promise<void> {
  // Fetch logic
}
</script>
```

---

## 34.3 Typing Component Props

Vue 3 provides `defineProps` for type-safe props with full TypeScript support.

### Basic Props with defineProps

```vue
<!-- UserCard.vue -->
<script setup lang="ts">
// Define props with interface
interface Props {
  name: string
  email: string
  age?: number
  isVerified?: boolean
  role?: 'admin' | 'user' | 'guest'
}

// With defaults using withDefaults
const props = withDefaults(defineProps<Props>(), {
  age: 0,
  isVerified: false,
  role: 'user'
})

// Access props (reactive but readonly)
console.log(props.name)

// Computed based on props
const displayRole = computed(() => {
  return props.role?.toUpperCase() ?? 'USER'
})
</script>

<template>
  <div class="user-card">
    <h3>{{ name }} <span v-if="isVerified">✓</span></h3>
    <p>{{ email }}</p>
    <p v-if="age > 0">Age: {{ age }}</p>
    <span class="badge">{{ displayRole }}</span>
  </div>
</template>
```

**Usage:**
```vue
<script setup lang="ts">
import UserCard from './UserCard.vue'
</script>

<template>
  <!-- TypeScript checks all props -->
  <UserCard 
    name="John Doe"
    email="john@example.com"
    :age="30"
    :is-verified="true"
    role="admin"
  />
  
  <!-- Error: Missing required prop 'name' -->
  <!-- <UserCard email="test@test.com" /> -->
  
  <!-- Error: Type 'number' is not assignable to type 'string' -->
  <!-- <UserCard name="123" email="test@test.com" age="30" /> -->
</template>
```

### Complex Props

```vue
<!-- DataTable.vue -->
<script setup lang="ts">
import type { PropType } from 'vue'

// Generic type for table data
interface TableColumn<T> {
  key: keyof T
  label: string
  sortable?: boolean
  formatter?: (value: T[keyof T], row: T) => string
}

interface TableProps<T> {
  data: T[]
  columns: TableColumn<T>[]
  loading?: boolean
  selectable?: boolean
  pageSize?: number
}

// Define generic props
const props = defineProps<TableProps<any>>()

// Or with specific type
interface User {
  id: number
  name: string
  email: string
  status: 'active' | 'inactive'
}

const userProps = defineProps<{
  data: User[]
  columns: TableColumn<User>[]
}>()

// Events
const emit = defineEmits<{
  (e: 'select', row: User): void
  (e: 'sort', key: keyof User, order: 'asc' | 'desc'): void
  (e: 'update:page', page: number): void
}>()

const handleSort = (column: TableColumn<User>) => {
  if (!column.sortable) return
  emit('sort', column.key, 'asc')
}
</script>
```

### Props Destructuring (Vue 3.3+)

```vue
<script setup lang="ts">
interface Props {
  title: string
  count?: number
  items: string[]
}

// With Vue 3.3+, can destructure with reactivity
const { title, count = 0, items } = defineProps<Props>()

// Note: Destructured props are NOT reactive unless using toRefs
// For reactivity, use props.title or convert to refs

import { toRefs } from 'vue'
const props = defineProps<Props>()
const { title: reactiveTitle } = toRefs(props)
</script>
```

---

## 34.4 Typing Component Emits

`defineEmits` provides type-safe event emission with full autocomplete and validation.

### Basic Emits

```vue
<!-- Counter.vue -->
<script setup lang="ts">
// Define emits with type literal
const emit = defineEmits<{
  increment: [value: number]
  decrement: [value: number]
  reset: []
  update: [value: number, timestamp: Date]
}>()

const handleIncrement = () => {
  emit('increment', 1)
}

const handleUpdate = (val: number) => {
  emit('update', val, new Date())
}
</script>

<template>
  <button @click="handleIncrement">+</button>
</template>
```

**Parent Component:**
```vue
<script setup lang="ts">
import Counter from './Counter.vue'

const handleIncrement = (value: number) => {
  console.log(`Incremented by ${value}`) // value is typed as number
}

const handleUpdate = (value: number, timestamp: Date) => {
  console.log(`Updated to ${value} at ${timestamp}`)
}
</script>

<template>
  <Counter 
    @increment="handleIncrement"
    @update="handleUpdate"
  />
</template>
```

### v-model with defineModel (Vue 3.3+)

```vue
<!-- CustomInput.vue -->
<script setup lang="ts">
// Define v-model with types
const modelValue = defineModel<string>({ required: true })

// Multiple v-models
const title = defineModel<string>('title', { default: '' })
const count = defineModel<number>('count', { default: 0 })

// With modifiers
const search = defineModel<string>('modelValue', {
  set(value) {
    return value.trim().toLowerCase()
  }
})
</script>

<template>
  <input 
    v-model="modelValue"
    type="text"
  />
  
  <input v-model="title" placeholder="Title" />
  <input v-model.number="count" type="number" />
</template>
```

**Usage:**
```vue
<script setup lang="ts">
import { ref } from 'vue'
import CustomInput from './CustomInput.vue'

const message = ref<string>('') // Type checked
const pageTitle = ref('')
const counter = ref(0)
</script>

<template>
  <CustomInput 
    v-model="message"
    v-model:title="pageTitle"
    v-model:count="counter"
  />
</template>
```

---

## 34.5 Type-Safe Provide/Inject

Dependency injection in Vue uses `provide` and `inject` with TypeScript support.

### Basic Provide/Inject

```typescript
// Injection keys (symbol-based for uniqueness)
export const UserKey: InjectionKey<{ 
  user: Ref<User | null>
  login: (email: string, password: string) => Promise<void>
  logout: () => void
}> = Symbol('user')

// Provider component
<script setup lang="ts">
import { provide, ref } from 'vue'
import { UserKey } from './injectionKeys'
import type { User } from './types'

const user = ref<User | null>(null)

const login = async (email: string, password: string): Promise<void> => {
  // API call
  const response = await fetch('/api/login', {
    method: 'POST',
    body: JSON.stringify({ email, password })
  })
  user.value = await response.json()
}

const logout = (): void => {
  user.value = null
}

provide(UserKey, {
  user,
  login,
  logout
})
</script>

// Consumer component
<script setup lang="ts">
import { inject } from 'vue'
import { UserKey } from './injectionKeys'

const userContext = inject(UserKey)

if (!userContext) {
  throw new Error('UserContext must be provided')
}

const { user, login, logout } = userContext

// Or with default value
const { user: injectedUser } = inject(UserKey, { 
  user: ref(null), 
  login: async () => {}, 
  logout: () => {} 
})
</script>

<template>
  <div v-if="user">
    <p>Welcome, {{ user.name }}</p>
    <button @click="logout">Logout</button>
  </div>
  <div v-else>
    <button @click="login('test@test.com', 'password')">Login</button>
  </div>
</template>
```

### Composable Pattern

```typescript
// composables/useUser.ts
import { provide, inject, ref, type Ref, type InjectionKey } from 'vue'

interface User {
  id: string
  name: string
  email: string
}

interface UserContext {
  user: Ref<User | null>
  isAuthenticated: Ref<boolean>
  login: (email: string, password: string) => Promise<void>
  logout: () => Promise<void>
}

const UserInjectionKey: InjectionKey<UserContext> = Symbol('user')

export function provideUser(): UserContext {
  const user = ref<User | null>(null)
  const isAuthenticated = ref(false)

  const login = async (email: string, password: string): Promise<void> => {
    const response = await fetch('/api/login', {
      method: 'POST',
      headers: { 'Content-Type': 'application/json' },
      body: JSON.stringify({ email, password })
    })
    
    if (!response.ok) throw new Error('Login failed')
    
    user.value = await response.json()
    isAuthenticated.value = true
  }

  const logout = async (): Promise<void> => {
    await fetch('/api/logout', { method: 'POST' })
    user.value = null
    isAuthenticated.value = false
  }

  const context: UserContext = {
    user,
    isAuthenticated,
    login,
    logout
  }

  provide(UserInjectionKey, context)
  return context
}

export function useUser(): UserContext {
  const context = inject(UserInjectionKey)
  
  if (!context) {
    throw new Error('useUser must be used within a component where provideUser was called')
  }
  
  return context
}
```

---

## 34.6 Typing Pinia Stores

Pinia is the recommended state management solution for Vue 3 with excellent TypeScript support.

### Store Definition

```typescript
// stores/user.ts
import { defineStore } from 'pinia'
import { ref, computed } from 'vue'

// Types
export interface User {
  id: string
  name: string
  email: string
  role: 'admin' | 'user' | 'guest'
  preferences: {
    theme: 'light' | 'dark'
    language: string
  }
}

// Store setup function
export const useUserStore = defineStore('user', () => {
  // State
  const user = ref<User | null>(null)
  const isLoading = ref(false)
  const error = ref<string | null>(null)

  // Getters (computed)
  const isAuthenticated = computed<boolean>(() => user.value !== null)
  const isAdmin = computed<boolean>(() => user.value?.role === 'admin')
  const fullName = computed<string>(() => {
    return user.value?.name ?? 'Guest'
  })

  // Actions
  async function login(email: string, password: string): Promise<void> {
    isLoading.value = true
    error.value = null
    
    try {
      const response = await fetch('/api/login', {
        method: 'POST',
        headers: { 'Content-Type': 'application/json' },
        body: JSON.stringify({ email, password })
      })
      
      if (!response.ok) throw new Error('Login failed')
      
      user.value = await response.json()
    } catch (err) {
      error.value = err instanceof Error ? err.message : 'Unknown error'
      throw err
    } finally {
      isLoading.value = false
    }
  }

  async function logout(): Promise<void> {
    await fetch('/api/logout', { method: 'POST' })
    user.value = null
  }

  function updatePreferences(prefs: Partial<User['preferences']>): void {
    if (user.value) {
      user.value.preferences = { ...user.value.preferences, ...prefs }
    }
  }

  return {
    user,
    isLoading,
    error,
    isAuthenticated,
    isAdmin,
    fullName,
    login,
    logout,
    updatePreferences
  }
})
```

**Usage in Components:**
```vue
<script setup lang="ts">
import { useUserStore } from '@/stores/user'
import { storeToRefs } from 'pinia'

const userStore = useUserStore()

// Use storeToRefs for reactive state
const { user, isAuthenticated, isAdmin } = storeToRefs(userStore)

// Actions can be destructured directly
const { login, logout } = userStore

// Access computed properties
console.log(userStore.fullName)

// Type-safe actions
const handleLogin = async () => {
  try {
    await login('user@example.com', 'password')
  } catch (error) {
    console.error(error)
  }
}
</script>

<template>
  <div>
    <p v-if="isAuthenticated">Welcome, {{ fullName }}</p>
    <button v-if="!isAuthenticated" @click="handleLogin">Login</button>
    <button v-else @click="logout">Logout</button>
    
    <div v-if="isAdmin">
      <h3>Admin Panel</h3>
      <!-- Admin only content -->
    </div>
  </div>
</template>
```

### Store with Options API Style

```typescript
// stores/counter.ts
import { defineStore } from 'pinia'

interface CounterState {
  count: number
  name: string
}

export const useCounterStore = defineStore('counter', {
  state: (): CounterState => ({
    count: 0,
    name: 'Counter'
  }),
  
  getters: {
    doubleCount: (state): number => state.count * 2,
    displayName: (state): string => `${state.name}: ${state.count}`
  },
  
  actions: {
    increment(amount: number = 1): void {
      this.count += amount
    },
    
    async asyncIncrement(delay: number = 1000): Promise<void> {
      return new Promise((resolve) => {
        setTimeout(() => {
          this.increment()
          resolve()
        }, delay)
      })
    }
  }
})
```

---

## 34.7 Script Setup Syntax

`<script setup>` is the recommended syntax for Vue 3 components, providing better TypeScript inference and less boilerplate.

### Basic Script Setup

```vue
<script setup lang="ts">
// Imports are automatically available in template
import { ref, computed } from 'vue'
import type { User } from '@/types'

// Reactive state
const count = ref<number>(0)
const user = ref<User | null>(null)

// Computed
const doubleCount = computed(() => count.value * 2)

// Methods
const increment = () => {
  count.value++
}

// Lifecycle
onMounted(() => {
  console.log('Component mounted')
})
</script>

<template>
  <div>
    <p>Count: {{ count }}</p>
    <p>Double: {{ doubleCount }}</p>
    <button @click="increment">+</button>
  </div>
</template>
```

### Exposing Component API

```vue
<script setup lang="ts">
import { ref } from 'vue'

const count = ref(0)
const increment = () => count.value++

// Expose to parent via template ref
defineExpose({
  count,
  increment,
  reset: () => count.value = 0
})
</script>

<template>
  <div>Count: {{ count }}</div>
</template>
```

**Parent Access:**
```vue
<script setup lang="ts">
import { ref, onMounted } from 'vue'
import Counter from './Counter.vue'

const counterRef = ref<InstanceType<typeof Counter> | null>(null)

onMounted(() => {
  // Type-safe access to exposed properties
  console.log(counterRef.value?.count)
  counterRef.value?.increment()
})
</script>

<template>
  <Counter ref="counterRef" />
</template>
```

---

## 34.8 Generic Components in Vue

Vue 3.3+ supports generic components using the generic attribute.

### Generic Component Definition

```vue
<!-- GenericList.vue -->
<script setup lang="ts" generic="T extends { id: string | number }">
import { computed } from 'vue'

// Props using generic type
const props = defineProps<{
  items: T[]
  selected?: T['id']
  keyExtractor?: (item: T) => string
}>()

const emit = defineEmits<{
  (e: 'select', item: T): void
  (e: 'update:selected', id: T['id']): void
}>()

// Computed with generic type
const selectedItem = computed(() => 
  props.items.find(item => item.id === props.selected)
)

const handleSelect = (item: T) => {
  emit('select', item)
  emit('update:selected', item.id)
}

// Generic slot props
defineSlots<{
  default(props: { item: T; index: number; isSelected: boolean }): any
  header?: (props: { count: number }) => any
  empty?: () => any
}>()
</script>

<template>
  <div class="generic-list">
    <slot name="header" :count="items.length">
      <h3>Items ({{ items.length }})</h3>
    </slot>
    
    <div v-if="items.length === 0">
      <slot name="empty">
        <p>No items</p>
      </slot>
    </div>
    
    <ul v-else>
      <li 
        v-for="(item, index) in items" 
        :key="item.id"
        :class="{ selected: item.id === selected }"
        @click="handleSelect(item)"
      >
        <slot 
          :item="item" 
          :index="index"
          :is-selected="item.id === selected"
        />
      </li>
    </ul>
  </div>
</template>
```

**Usage:**
```vue
<script setup lang="ts">
import GenericList from './GenericList.vue'

interface Product {
  id: string
  name: string
  price: number
}

const products = ref<Product[]>([
  { id: '1', name: 'Laptop', price: 999 },
  { id: '2', name: 'Mouse', price: 29 }
])

const selectedProduct = ref<string | undefined>()
</script>

<template>
  <GenericList 
    v-model:selected="selectedProduct"
    :items="products"
  >
    <template #default="{ item, isSelected }">
      <div :class="{ 'bg-blue': isSelected }">
        <strong>{{ item.name }}</strong>
        <span>${{ item.price }}</span>
      </div>
    </template>
  </GenericList>
</template>
```

---

## 34.9 Type-Safe Router (Vue Router 4)

Vue Router 4 provides TypeScript support for typed routes and navigation guards.

### Router Setup with Types

```typescript
// router/index.ts
import { createRouter, createWebHistory } from 'vue-router'
import type { RouteRecordRaw } from 'vue-router'

// Extend route meta type
declare module 'vue-router' {
  interface RouteMeta {
    requiresAuth?: boolean
    roles?: string[]
    title?: string
    layout?: 'default' | 'auth' | 'admin'
  }
}

// Route definitions
const routes: RouteRecordRaw[] = [
  {
    path: '/',
    name: 'Home',
    component: () => import('@/views/HomeView.vue')
  },
  {
    path: '/about',
    name: 'About',
    component: () => import('@/views/AboutView.vue'),
    meta: { title: 'About Us' }
  },
  {
    path: '/users/:id',
    name: 'UserDetail',
    component: () => import('@/views/UserDetailView.vue'),
    props: true,
    meta: { requiresAuth: true }
  },
  {
    path: '/admin',
    name: 'Admin',
    component: () => import('@/views/AdminView.vue'),
    meta: { 
      requiresAuth: true, 
      roles: ['admin'],
      layout: 'admin'
    }
  },
  {
    path: '/login',
    name: 'Login',
    component: () => import('@/views/LoginView.vue'),
    meta: { layout: 'auth' }
  },
  {
    path: '/:pathMatch(.*)*',
    name: 'NotFound',
    component: () => import('@/views/NotFoundView.vue')
  }
]

const router = createRouter({
  history: createWebHistory(import.meta.env.BASE_URL),
  routes
})

// Navigation guards with types
import { useUserStore } from '@/stores/user'

router.beforeEach((to, from, next) => {
  const userStore = useUserStore()
  
  // Type-safe meta access
  if (to.meta.requiresAuth && !userStore.isAuthenticated) {
    next({ name: 'Login', query: { redirect: to.fullPath } })
    return
  }
  
  if (to.meta.roles && !to.meta.roles.includes(userStore.user?.role || '')) {
    next({ name: 'Home' })
    return
  }
  
  next()
})

router.afterEach((to) => {
  // Type-safe meta access
  document.title = to.meta.title || 'My App'
})

export default router
```

### Typed Navigation

```vue
<script setup lang="ts">
import { useRouter, useRoute } from 'vue-router'

const router = useRouter()
const route = useRoute()

// Type-safe route params
const userId = computed(() => route.params.id as string)

// Type-safe navigation
const goToUser = async (id: string) => {
  await router.push({ name: 'UserDetail', params: { id } })
}

const goToHome = () => {
  router.push({ name: 'Home' })
}

// With query params
const search = () => {
  router.push({
    name: 'Search',
    query: { q: 'typescript', page: '1' }
  })
}

// Replace current history entry
const replaceRoute = () => {
  router.replace({ name: 'Login' })
}
</script>
```

### Route Params Typing

```vue
<!-- UserDetailView.vue -->
<script setup lang="ts">
import { useRoute } from 'vue-router'
import { computed, watch } from 'vue'

const route = useRoute()

// Type assertion for params
const userId = computed(() => route.params.id as string)

// Or with validation
const validatedUserId = computed(() => {
  const id = route.params.id
  if (typeof id !== 'string' || !/^\d+$/.test(id)) {
    throw new Error('Invalid user ID')
  }
  return id
})

// Watch for param changes
watch(() => route.params.id, (newId, oldId) => {
  if (newId !== oldId) {
    fetchUser(newId as string)
  }
})

async function fetchUser(id: string): Promise<void> {
  // Fetch user data
}
</script>
```

---

## 34.10 Chapter Summary and Exercises

### Chapter Summary

This chapter covered TypeScript integration with Vue.js:

**Key Concepts:**

1. **Setup**: Vite provides the best Vue + TypeScript experience. `env.d.ts` handles type declarations for `.vue` files and environment variables.

2. **Composition API**: `ref<T>()` and `reactive<T>()` provide typed reactive state. `computed()` infers types automatically. Prefer `ref` for primitives and `reactive` for objects.

3. **Props**: `defineProps<Interface>()` with TypeScript interfaces provides compile-time prop validation. `withDefaults()` adds default values. Vue 3.3+ supports props destructuring.

4. **Emits**: `defineEmits<{}>()` uses TypeScript literal types for type-safe event definitions. `defineModel()` (Vue 3.3+) provides typed two-way binding.

5. **Provide/Inject**: Use `InjectionKey<T>` for type-safe dependency injection. Always check for undefined or provide default values.

6. **Pinia**: The recommended state management solution with excellent TypeScript support. Use `storeToRefs()` for reactive state extraction. Setup stores with `ref`/`computed` provide better type inference than Options API style.

7. **Script Setup**: The recommended syntax for Vue 3 components, providing better TypeScript inference and less boilerplate. `defineExpose()` controls public component API.

8. **Generic Components**: Vue 3.3+ supports `generic="T"` attribute for creating reusable, type-safe components that work with any data type.

9. **Router**: Vue Router 4 provides typed navigation guards and route meta fields. Extend `RouteMeta` interface for custom meta properties.

### Practical Exercises

**Exercise 1: Type-Safe Form Component**

Create a generic form component:
- Props: `modelValue: T`, `schema: ValidationSchema<T>`, `fields: FormField<T>[]`
- Emits: `update:modelValue`, `submit`, `validate`
- Use generics to ensure field names match object keys
- Implement validation with TypeScript-discriminated unions for error types
- Support different input types (text, number, select, checkbox) with proper type inference

**Exercise 2: Composable Library**

Build a set of typed composables:
- `useFetch<T>(url, options)` with automatic type inference from response
- `useLocalStorage<T>(key, defaultValue)` with JSON serialization types
- `usePagination<T>(items, pageSize)` returning typed current page items
- `useValidation<T>(schema)` returning typed errors object
- Ensure all composables have proper generic constraints and return types

**Exercise 3: Pinia Store with Modules**

Create an e-commerce store:
- `useProductStore` with `Product` type, filtering by category with typed filters
- `useCartStore` with `CartItem` type, computed totals with proper number types
- `useOrderStore` with async actions, discriminated unions for order status
- Implement store plugins for persistence with typed storage keys
- Add store-to-store communication with proper typing

**Exercise 4: Generic Table with Sorting**

Build a `DataTable<T>` component:
- Props: `data: T[]`, `columns: ColumnDef<T>[]`, `sortable?: boolean`
- ColumnDef should include `key: keyof T`, `label: string`, `sortable?: boolean`, `formatter?: (value: T[K]) => string`
- Implement generic sorting function that respects key types
- Add row selection with `selected: T['id'][]` (assuming T has id)
- Support custom cell rendering with scoped slots preserving T type

**Exercise 5: Type-Safe Router Guards**

Implement authentication system:
- Extend `RouteMeta` with `permissions: string[]`
- Create `useAuth()` composable returning `user: Ref<User | null>`, `hasPermission(permission: string): boolean`
- Implement navigation guard checking permissions against route meta
- Create typed `router.push()` wrapper that validates route names and params
- Add breadcrumb system with typed route hierarchy

**Exercise 6: Component Library**

Create a typed UI library:
- `Button` component with variant union types, proper disabled states
- `Modal` component with generic `result: T` for close events
- `Select<T>` component with `options: T[]`, `value: T`, `labelKey: keyof T`
- `Tabs` compound component with typed active tab state
- Ensure all components emit properly typed events and support v-model

**Exercise 7: API Integration**

Build a type-safe API client:
- Define API endpoints as const object with typed paths
- Create `useApi()` composable with methods `get<T>`, `post<T>`, `put<T>`, `delete<T>`
- Implement request/response interceptors with typed error handling
- Use Zod for runtime validation with inferred TypeScript types
- Add automatic token refresh with typed auth headers

**Exercise 8: Full Application**

Build a Task Management app:
- TypeScript interfaces for Task, Project, User with relations
- Pinia stores with typed state, getters, actions
- Vue Router with typed routes and navigation guards
- Generic components for lists, forms, modals
- API integration with type-safe fetch wrapper
- Form validation with TypeScript-discriminated error types

### Additional Resources

- **Vue TypeScript Documentation**: https://vuejs.org/guide/typescript/overview.html
- **Vue Macros**: https://vue-macros.sxzz.moe/ - Advanced TypeScript macros for Vue
- **VueUse**: https://vueuse.org/ - Collection of essential Vue Composition Utilities with TypeScript
- **Pinia Documentation**: https://pinia.vuejs.org/core-concepts/
- **Vue Router TypeScript**: https://router.vuejs.org/guide/advanced/typed-routes.html

---

## Coming Up Next: Chapter 35 - TypeScript with Angular

In the next chapter, we will explore TypeScript with Angular:

- Angular's built-in TypeScript architecture
- Setting up Angular projects with strict mode
- Typing components with decorators (@Component, @Input, @Output)
- Dependency Injection and service typing
- RxJS integration with TypeScript
- Forms: Template-driven and Reactive forms with types
- HTTP Client with typed requests and responses
- Angular CLI and TypeScript configuration
- Advanced decorators and metadata

Understanding TypeScript with Angular is essential as Angular was built with TypeScript from the ground up, offering the most comprehensive type safety and tooling integration of any major frontend framework.

<div style='width:100%; display:flex; justify-content:space-between; align-items:center; margin: 1em 0;'>
  <a href='33. typescript_with_nodejs.ipynb' style='font-weight:bold; font-size:1.05em;'>&larr; Previous</a>
  <a href='../TOC.md' style='font-weight:bold; font-size:1.05em; text-align:center;'>Table of Contents</a>
  <a href='35. typescript_with_angular.ipynb' style='font-weight:bold; font-size:1.05em;'>Next &rarr;</a>
</div>
